# Evaluation of postprocessing approaches

## Overview
In general, nuclei instance segmentation can be formulated as a classification or a regression problem. Hence, the model's objective is to learn a segmentation mask or a distance map for the classification and regression problem, respectively. In a segmentation mask, each pixel is assigned to a certain class. Such a class might be background, nucleus, nucleus center or nucleus border. In a binary segmentation mask, each pixel represents the probability of the pixel belonging to one of the two classes (e.g. nucleus vs. background). In a distance map, each pixel stats the minimal distance between the pixel and a pixel of a certain class. The chessboard distance is commonly used as distance measure. However, other distance measures like the Manhattan or Euclidian distance could be used as well. Moreover, the distance might be calculated along a certain direction and the distance map normalized. A collection of possible design choices for segmentation masks and distance maps is presented below.

**Segmentation mask S**
- Classes: Background, nucleus, nucleus contour, nucleus border etc.

**Distance map D**
- Distance measure: Manhattan distance, Chessboard distance, Euclidian distance etc.
- Direction: None, vertical, horizontal, etc.
- Normalization: None, per-instance-normalized, per-image-normalized, per-dataset-normalized
- Classes: Background, nucleus, nucleus center, nucleus contour etc.

The individual design choices and the combination of multiple segmentation masks or distances maps allow for different postprocessing strategies.
- S(nucleus): Morphological operations (opening etc.), watershed, clustering, (MDL-constrained) periodic B-splines, Leonidas Spaß etc.
- S(nucleus), S(contour): S(nucleus) & ~S(contours) -> watershed (-> dilation/h-maxima reconstruction)

## Aim
Here, a qualitative and quantitative evaluation of the following postprocessing approaches is presented:
- Standard: S(nucleus) & ~S(contours)
- Naylor et al. 2019: D(background) using dynamic watershed
- HoVer-Net et al. 2019: D_v and D_h

In [1]:
from data.MoNuSeg.data_module import MoNuSegDataModule
from evaluation.metrics import AJI, PQ
from postprocessing.segmentation import SegPostProcess, DistPostProcess, HVPostProcess
from data.MoNuSeg.ground_truth import NucleiInstances
from data.MoNuSeg.illustrator import Picture

In [2]:
data_module = MoNuSegDataModule(
    seg_masks=True,
    cont_masks=True,
    dist_maps=True,
    hv_maps=True,
    labels=False,
    train_transforms=None,
    data_root="datasets"
)
# data_module.prepare_data()
data_module.setup(stage="fit")
train_loader = data_module.train_dataloader()

In [3]:
# optn, dilate = "skimage", True
param, thresh = 3, 0
run_al_postprocess, run_dist_postprocess, run_hover_postprocess = False, False, True

aji_al, aji_dist, aji_hover = AJI(), AJI(), AJI()
pq_al, pq_dist, pq_hover = PQ(), PQ(), PQ()

al_postprocess = SegPostProcess()
dist_postprocess = DistPostProcess(param=param, thresh=thresh)
hover_postprocess = HVPostProcess()

for batch_num, (imgs, seg_masks, cont_masks, dist_maps, hv_maps, inst) in enumerate(train_loader):
    if run_al_postprocess:
        inst_al = al_postprocess(seg_masks, cont_masks)
        aji_al.update(inst_al, inst)
        pq_al.update(inst_al, inst)

    if run_dist_postprocess:
        inst_dist = dist_postprocess(dist_maps)
        aji_dist.update(inst_dist, inst)
        pq_dist.update(inst_dist, inst)

    if run_hover_postprocess:
        inst_hover = hover_postprocess(hv_maps, seg_masks)
        aji_hover.update(inst_hover, inst)
        pq_hover.update(inst_hover, inst)

if run_al_postprocess:
    aji_score_al = aji_al.compute()
    pq_score_al = pq_al.compute()
    print("######### AL #########")
    # print(f"Option: {optn}")
    # print(f"Dilate: {dilate}")
    print(f"AJI: {aji_score_al['AJI']}")
    print(f"SQ: {pq_score_al['SQ']}")
    print(f"DQ: {pq_score_al['DQ']}")
    print(f"PQ: {pq_score_al['PQ']}")

if run_dist_postprocess:
    aji_score_dist = aji_dist.compute()
    pq_score_dist = pq_dist.compute()
    print("######### DIST #########")
    print(f"Parameter: {param}")
    print(f"Threshold: {thresh}")
    print(f"AJI: {aji_score_dist['AJI']}")
    print(f"SQ: {pq_score_dist['SQ']}")
    print(f"DQ: {pq_score_dist['DQ']}")
    print(f"PQ: {pq_score_dist['PQ']}")

if run_hover_postprocess:
    aji_score_hover = aji_hover.compute()
    pq_score_hover = pq_hover.compute()
    print("######### HoVer #########")
    print(f"AJI: {aji_score_hover['AJI']}")
    print(f"SQ: {pq_score_hover['SQ']}")
    print(f"DQ: {pq_score_hover['DQ']}")
    print(f"PQ: {pq_score_hover['PQ']}")

######### HoVer #########
AJI: 0.8732954263687134
SQ: 0.9730230569839478
DQ: 0.9455816745758057
PQ: 0.9200727939605713


######### HoVer #########
No noise suppression!
sobel_h = cv2.Sobel(h_map, cv2.CV_64F, 1, 0, ksize=5)
sobel_v = cv2.Sobel(v_map, cv2.CV_64F, 0, 1, ksize=5)
AJI: 0.8732954263687134
SQ: 0.9730230569839478
DQ: 0.9455816745758057
PQ: 0.9200727939605713
######### HoVer #########
No noise suppression!
sobel_h = farid_h(h_map, mask=seg_mask, mode='full', boundary='fill', fillvalue=0)
sobel_v = farid_v(v_map, mask=seg_mask, mode='full', boundary='fill', fillvalue=0)
AJI: 0.8898743987083435
SQ: 0.9690139293670654
DQ: 0.9524402022361755
PQ: 0.9229277968406677
######### HoVer #########
No noise suppression!
energy_landscape = (1.0 - combined)
AJI: 0.8918822407722473
SQ: 0.9797292947769165
DQ: 0.9530668258666992
PQ: 0.9337474703788757
######### HoVer #########
No noise suppression!
AJI: 0.8866761326789856
SQ: 0.9642669558525085
DQ: 0.9515339732170105
PQ: 0.917532742023468
######### AL #########
Minimal size: 0
AJI: 0.9586374163627625
SQ: 0.9694679379463196
DQ: 0.9623864889144897
PQ: 0.9330028295516968
######### AL #########
Minimal size: 15
AJI: 0.953377366065979
SQ: 0.9697842597961426
DQ: 0.9579931497573853
PQ: 0.9290466904640198
######### AL #########
Minimal size: 10
AJI: 0.9563164114952087
SQ: 0.9689503908157349
DQ: 0.9578863978385925
PQ: 0.9281443953514099
######### AL #########
Option: skimage
Dilate: False
AJI: 0.7602019906044006
SQ: 0.7199723720550537
DQ: 0.8247086405754089
PQ: 0.5937674641609192
######### AL #########
Option: skimage
Dilate: True
AJI: 0.9552018046379089
SQ: 0.9727380275726318
DQ: 0.9514009356498718
PQ: 0.925463855266571
######### AL #########
Option: custom
AJI: 0.7602019906044006
SQ: 0.7199727892875671
DQ: 0.8247086405754089
PQ: 0.5937677621841431
######### DIST #########
Parameter: 0
Threshold: 0
AJI: 0.8587603569030762
SQ: 0.9458383321762085
DQ: 0.9079502820968628
PQ: 0.8587741851806641
######### DIST #########
Parameter: 1
Threshold: 0
AJI: 0.9227789044380188
SQ: 0.9605621099472046
DQ: 0.9713776707649231
PQ: 0.933068573474884
######### DIST #########
Parameter: 2
Threshold: 0
AJI: 0.891636848449707
SQ: 0.9619731903076172
DQ: 0.9442554116249084
PQ: 0.9083483815193176
######### DIST #########
Parameter: 3
Threshold: 0
AJI: 0.8139533996582031
SQ: 0.9603028297424316
DQ: 0.8777669668197632
PQ: 0.8429220914840698
